# Prepare Data for Neural Retrieval

In [1]:
!pip install transformers
!pip3 install pickle5
!pip install torch
!pip install datasets

     |████████████████████████████████| 2.3MB 7.8MB/s 
     |████████████████████████████████| 3.3MB 41.5MB/s 
     |████████████████████████████████| 901kB 40.4MB/s 
     |████████████████████████████████| 133kB 8.1MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219283 sha256=1346771e41b4577a76c07b3ea749e95953038c4e91013d07b6f9391a1d42daea
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
     |████████████████████████████████| 245kB 7.6MB/s 
     |████████████████████████████████| 122kB 13.1MB/s 
     |████████████████████████████████| 245kB 13.4MB/s 


In [2]:
from google.colab import drive
import pickle5 as pickle
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch
from torch.utils.data import Dataset, IterableDataset, DataLoader
import random
import multiprocessing as mp
import datetime
import os
import random

## Load Files

In [3]:
def save_as_pickle(obj, filename):
    """
    save an object in a pickle file dump
    :param obj: object to dump
    :param filename: target file
    :return:
    """
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, protocol=pickle.HIGHEST_PROTOCOL)


def load_pickle(filename):
    """
    load an object from a given pickle file
    :param filename: source file
    :return: loaded object
    """
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [4]:
drive.mount('/content/drive', force_remount=True)  # use force_remount=True param after upload of new data

# file and folder names
ir_project_drive_folder = "Master/2 - FSS 2021/Information Retrieval/IR Projekt"  # TODO: geht das für alle?
# ir_project_drive_folder = "IR Projekt"  # TODO: geht das für alle?
full_ir_project_drive_folder = "/content/drive/My Drive/{}/data/wikipedia".format(ir_project_drive_folder)

# full_ir_project_drive_folder = '../data/wikipedia'
raw_folder = full_ir_project_drive_folder + '/raw'
nr_datasets_folder = full_ir_project_drive_folder + '/neural-retrieval-datasets'

# raw qa filenames
qa_wikipedia_verified_dev_filename = raw_folder + '/qa/verified-wikipedia-dev.json'
qa_wikipedia_dev_filename = raw_folder + '/qa/wikipedia-dev.json'
qa_wikipedia_test_without_answers_filename = raw_folder + '/qa/wikipedia-test-without-answers.json'
qa_wikipedia_train_filename = raw_folder + '/qa/wikipedia-train.json'

preprocessed_qa_wikipedia_dev_filename = full_ir_project_drive_folder + '/no-pron/preprocessed/qa/wikipedia-dev.pkl'


# preprocessed evidence files
wikipedia_evidence_file = raw_folder + '/wikipedia_evidence_dict.pkl'

# bm25 results
bm25_retrieved_train_TP_FP_filename = full_ir_project_drive_folder + "/neural-retrieval-datasets/bm25_tp_fp_train.pkl"
bm25_retrieved_dev_TP_FP_filename = full_ir_project_drive_folder + "/neural-retrieval-datasets/bm25_tp_fp_dev.pkl"
bm25_retrieved_dev_pl_map_filename = full_ir_project_drive_folder + "/neural-retrieval-datasets/dev_pl_map.pkl"

## neural retrieval
nr_train_relevant_set_filename = nr_datasets_folder + '/train_relevant'
nr_train_irrelevant_random_set_filename = nr_datasets_folder + '/train_irrelevant_random'
nr_train_irrelevant_hard_random_set_filename = nr_datasets_folder + '/train_irrelevant_hard_random'  # TODO: ERSTELLEN
nr_train_irrelevant_hard_max_set_filename = nr_datasets_folder + '/train_irrelevant_hard_max'

Mounted at /content/drive


In [ ]:
documents_dict = load_pickle(wikipedia_evidence_file)
documents_filename_list = list(documents_dict.keys())

In [5]:
bm25_retrieved_train_TP_FP = load_pickle(bm25_retrieved_train_TP_FP_filename)
bm25_retrieved_dev_TP_FP = load_pickle(bm25_retrieved_dev_TP_FP_filename)

In [ ]:
len(bm25_retrieved_train_TP_FP)

61888

In [9]:
for i,(question, data) in enumerate(bm25_retrieved_dev_TP_FP.items()):
  if i == 2:
    break
  print(question, data)

tc_33 {'matching_documents': ['Andrew_Lloyd_Webber.txt'], 'false positive document': ['Julian_Lloyd_Webber.txt', 'Adelphi_Theatre.txt', 'Sunset_Boulevard_(musical).txt', 'The_Phantom_of_the_Opera_(1986_musical).txt', "I_Don't_Know_How_to_Love_Him.txt", 'Michael_Ball_(singer).txt', 'Michael_Crawford.txt', 'Sarah_Brightman.txt', 'Marti_Webb.txt', 'Sing_(Gary_Barlow_song).txt', 'Joseph_and_the_Amazing_Technicolor_Dreamcoat.txt', 'Requiem_(Lloyd_Webber).txt', 'Whistle_Down_the_Wind_(musical).txt', "Don't_Cry_for_Me_Argentina.txt", 'Palace_Theatre,_London.txt', 'Tim_Rice.txt', 'Stephen_Ward_(musical).txt', 'Gielgud_Theatre.txt', 'Cameron_Mackintosh.txt', 'Starlight_Express.txt', 'How_Do_You_Solve_a_Problem_like_Maria_.txt', 'Evita_(musical).txt', 'Cats_(musical).txt', 'Glenn_Slater.txt', 'All_I_Ask_of_You.txt', 'Ben_Elton.txt', 'Madeleine_Gurdon.txt', 'The_Music_of_the_Night.txt', 'West_End_theatre.txt', 'Jesus_Christ_Superstar_(album).txt', 'The_South_Bank_Show.txt', 'Oh_What_a_Circus.txt'

## Init Tokenizer

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

## Create Datasets

In [ ]:
def create_samples(groups, mode, bm25_results=None):
  """
  creates sample based on dataset
  mode: categorical parameter with values
          relevant: creates relevant samples with entity pages
          irrelevant_random: creates irrelevant samples with random documents
          irrelevant_hard_random: creates irrelevant samples with random draws from top 50 ranked false positive BM25 results
          irrelevant_hard_max: creates irrelevant samples with 2 top from top 50 ranked false positive BM25 results
  bm25_results: dict of ranked false positives for every questions by BM25
  """
  questions = groups['Question']
  question_ids = groups['QuestionId']
  entity_pages_list = groups['EntityPages']

  result_questions = []
  result_relevant_documents = []

  if mode == 'relevant' or mode == 'irrelevant_random':
    # creation of irrelevant samples is in relevant entity pages loop 
    # -> create as many irrelevant samples as relevant samples
    for question, entity_pages in zip(questions, entity_pages_list):
      for entity_page in entity_pages:
        try:
          if mode == 'relevant':
            doc = documents_dict[entity_page['Filename'].replace(":","_").replace("?","_")]
          elif mode == 'irrelevant_random':
            doc = documents_dict[documents_filename_list[random.randint(0, len(documents_filename_list) - 1)].replace(":","_").replace("?","_")]
          result_relevant_documents.append(doc)
          result_questions.append(question)
        except KeyError:
          pass  # some files have different case sensitive key in qa file and in folder
  
  elif mode == 'irrelevant_hard_random':
    for question_id, question in zip(question_ids, questions):
      # draw to random for each question
      try:
        for i in range(0, 2):
          rand_index = random.randint(0, len(bm25_results[question_id]['false positive document']) - 1)
          result_relevant_documents.append(documents_dict[bm25_results[question_id]['false positive document'][rand_index]])
          result_questions.append(question)
      except (Exception) as e:
        pass

  elif mode == 'irrelevant_hard_max':
    for question_id, question in zip(question_ids, questions):
      for top_ranked_irrelevant_document in bm25_results[question_id]['false positive document'][:2]:
        result_relevant_documents.append(documents_dict[top_ranked_irrelevant_document])
        result_questions.append(question)
      
  
  if mode == 'relevant':
    label = 1
  else:
    label = 0
  labels = [label for i in range(len(result_questions))]
  
  return {"question": result_questions, "relevant_document": result_relevant_documents, 'label': labels}

def create_retrieval_dataset(path, mode, bm25_results=None, disk_path=None):
  dataset = load_dataset('json', data_files=[path], field='Data', keep_in_memory=False)["train"]
  pairs_set = dataset.map(create_samples, fn_kwargs={'mode': mode, 'bm25_results': bm25_results}, 
              batched=True, remove_columns=dataset.column_names, batch_size=32)
  tokenized_set = pairs_set.map(lambda examples: tokenizer(examples['question'], examples['relevant_document'], truncation=True, padding="max_length"),
                                batched=True)
  print(tokenized_set)
  if disk_path is not None:
    tokenized_set.save_to_disk(disk_path)
    print('saved as', disk_path)
  return tokenized_set

In [ ]:
# train_data_relevant = create_retrieval_dataset(qa_wikipedia_train_filename, 'relevant', disk_path=nr_train_relevant_set_filename)
# train_data_irrelevant = create_retrieval_dataset(qa_wikipedia_train_filename, 'irrelevant_random', disk_path=nr_train_irrelevant_set_filename)
# train_data_irrelevant_hard_max = create_retrieval_dataset(qa_wikipedia_train_filename, 'irrelevant_hard_max', bm25_results=bm25_retrieved_train_TP_FP, disk_path=nr_train_irrelevant_hard_max_set_filename)
train_data_irrelevant_hard_random = create_retrieval_dataset(qa_wikipedia_train_filename, 'irrelevant_hard_random', bm25_results=bm25_retrieved_train_TP_FP, disk_path=nr_train_irrelevant_hard_random_set_filename)

# dev_data_relevant = create_retrieval_dataset(qa_wikipedia_dev_filename, 'relevant', disk_path=nr_dev_relevant_set_filename)
# dev_data_irrelevant = create_retrieval_dataset(qa_wikipedia_dev_filename, 'irrelevant_random', disk_path=nr_dev_irrelevant_set_filename)


Using custom data configuration default-e42834cb7c019793
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-e42834cb7c019793/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


In [ ]:
dev_data_irrelevant_smart = create_retrieval_dataset(qa_wikipedia_dev_filename, 'irrelevant_smart', bm25_results=bm25_retrieved_dev_TP_FP, disk_path=nr_dev_irrelevant_smart_set_filename)

Using custom data configuration default-35fd0fffa233a823


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-35fd0fffa233a823/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.



Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'question', 'relevant_document'],
    num_rows: 15986
})
saved as /content/drive/My Drive/Master/2 - FSS 2021/Information Retrieval/IR Projekt/data/wikipedia/neural-retrieval-datasets/dev_irrelevant_smart


In [ ]:
train_data_irrelevant = create_retrieval_dataset(qa_wikipedia_train_filename, relevant=False, disk_path=nr_train_irrelevant_set_filename)

Using custom data configuration default-e42834cb7c019793
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-e42834cb7c019793/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)



Dataset({
    features: ['attention_mask', 'input_ids', 'question', 'relevant_document'],
    num_rows: 110647
})
saved as /content/drive/My Drive/Master/2 - FSS 2021/Information Retrieval/IR Projekt/data/wikipedia/neural-retrieval-datasets/train_irrelevant


In [ ]:
loaded = Dataset.load_from_disk(nr_dev_irrelevant_set_filename)
print(loaded)

Dataset({
    features: ['attention_mask', 'input_ids', 'question', 'relevant_document'],
    num_rows: 14229
})


## Prepare Devset for Pytorch Lightning MAP

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

In [ ]:
dev_bm25 = load_pickle(bm25_retrieved_dev_TP_FP_filename)
dev_queries = load_pickle(preprocessed_qa_wikipedia_dev_filename)
document_limit = 50
relevant_keys = list(dev_bm25.keys())[:100]
relevant_keys.sort()
dev_queries.sort(key=lambda qa_pair: qa_pair['QuestionId'])

In [ ]:
indexes = []
targets = []

docs = []
queries = []

i = 0
for qa_pair in dev_queries:
  question_id = qa_pair['QuestionId']
  if question_id in relevant_keys:
    matching_documents = dev_bm25[question_id]['matching_documents']
    number_false_positives = document_limit - len(matching_documents)
    false_positive_documents = dev_bm25[question_id]['false positive document'][:number_false_positives]
    

    indexes.extend([i for x in range(len(matching_documents) + number_false_positives)])
    targets.extend([True for x in range(len(matching_documents))])
    targets.extend([False for x in range(number_false_positives)])

    for doc in matching_documents:
      docs.append(documents_dict[doc])
      queries.append(qa_pair['Question'])    
    for doc in false_positive_documents:
      docs.append(documents_dict[doc])
      queries.append(qa_pair['Question'])

    i += 1

encoding = tokenizer(queries, docs, truncation=True, padding="max_length", return_tensors='pt') #, return_token_type_ids=False
encoding

{'input_ids': tensor([[    0, 12375,    21,  ...,    23, 22226,     2],
        [    0, 12375,    21,  ...,   380,  2441,     2],
        [    0, 12375,    21,  ...,   380,  2441,     2],
        ...,
        [    0,  2264,    21,  ...,  2176,  3225,     2],
        [    0,  2264,    21,  ...,   149, 11775,     2],
        [    0,  2264,    21,  ...,    64,   293,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
save_as_pickle((indexes, targets, encoding), bm25_retrieved_dev_pl_map_filename)